In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component,
                        OutputPath,
                        InputPath)
import google.cloud.aiplatform as aip

### Pipeline config

In [2]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "assignment1-402316"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2023_group1"
# image registry location
IMAGE_REG = "image-repo-group1"

In [4]:
@dsl.container_component
def toxic_data_ingestion(project: str, bucket: str, data_file_name: str,  features: Output[Dataset]):

    return dsl.ContainerSpec(
        image=f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REG}/toxic-data-ingestor:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id',project,'--bucket',bucket,'--file_name',data_file_name,'--feature_path', features.path])

In [16]:
@dsl.container_component
def toxic_data_cleaner(features: Input[Dataset], cleaned_data: Output[Dataset]):

    return dsl.ContainerSpec(
        image=f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REG}/toxic-data-cleaner:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--raw_data_path',features.path,
              '--cleaned_data_path', cleaned_data.path
             ]
    )

In [7]:
@dsl.container_component
def toxic_train_test_split(cleaned_data: Input[Dataset], train: Output[Dataset], test: Output[Dataset]):

    return dsl.ContainerSpec(
        image=f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REG}/toxic-train-test-split:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--clean_data_path',cleaned_data.path,
              '--train_path', train.path,
              '--test_path', test.path
             ]
    )

In [17]:
@dsl.container_component
def multilabel_classifier_trainer(project: str, train: Input[Dataset], model_bucket: str,  metrics: OutputPath(str)):

    return dsl.ContainerSpec(
        image=f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REG}/toxic-multilabel-trainer:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id', project,
              '--train_path', train.path,
              '--model_repo',model_bucket,
              '--metrics_path', metrics
             ]
    )

In [10]:
@dsl.container_component
def toxic_predictor(project: str, test: Input[Dataset], model_bucket: str, metrics: OutputPath(str)):

    return dsl.ContainerSpec(
        image=f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REG}/toxic-predictor:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id', project,
              '--predict_data', test.path,
              '--model_repo',model_bucket,
              '--metrics_path', metrics,
              '--validation_data'
             ]
    )

### Define the pipeline

In [20]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="toxic-predictor-mlp")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str):

    # The first step
    di_op = toxic_data_ingestion(
        project=project_id,
        bucket=data_bucket,
        data_file_name=trainset_filename
    )

    # The second step
    dc_op = toxic_data_cleaner(
        features=di_op.outputs['features']
    )

    # The third step
    tts_op = toxic_train_test_split(
        cleaned_data=dc_op.outputs['cleaned_data']
    )

    # The fourth step
    mc_op = multilabel_classifier_trainer(
        project=project_id,
        train=tts_op.outputs['train'],
        model_bucket=model_repo
    )

    # The fifth step
    tp_op = toxic_predictor(
        project=project_id,
        test=tts_op.outputs['test'],
        model_bucket=model_repo
    )

    # Ensure toxic_predictor only starts after multilabel_classifier is done
    tp_op.after(mc_op)

### Compile the Pipeline

In [21]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='toxic_predictor_mlp.yaml')

### Run the Pipeline

In [25]:
import google.cloud.aiplatform as aip
import os

#roman help: wat is dit / hebben wij dit gedaan? (GOOGLE_APPLICATION_CREDENTIALS)
# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "assignment1-402316-dc4baf177723.json"

aip.init(
    project=PROJECT_ID,
    location=REGION,
)

job = aip.PipelineJob(
    display_name="toxic-predictor-mlp-pipeline",
    template_path="toxic_predictor_mlp.yaml",
    enable_caching=False,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'project_id': PROJECT_ID,
        'data_bucket': 'data_de2023_group1',
        'trainset_filename': 'train.csv',
        'model_repo':'models_de2023_group1'
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/toxic-predictor-mlp-20231021204545?project=791882449847
PipelineJob projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545 current state:
3
PipelineJob projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545 current state:
3
PipelineJob projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545 current state:
3
PipelineJob projects/791882449847/locations/us-central1/pipelineJobs/toxic-predictor-mlp-20231021204545 current state:
3
PipelineJob projects/7918824498